<a href="https://colab.research.google.com/github/AICHUCKY/Ai-with-Chucky-Colab-Notebooks/blob/main/ACE_Step_1_5_Colab_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 ACE-Step v1.5 - Google Colab Launcher
### 🔴 Brought to you by [AI With Chucky](https://youtube.com/@AIWithChucky)

**Notes:**
- This requires a GPU runtime (T4 is usually sufficient).
- The first run will take a few minutes to download the models (approx 10GB).
- **Wait for the link ending in `gradio.live` to appear at the bottom.**

In [ ]:
# @title 1. Clone Repository & Patch Dependencies
import os
import sys

# 1. Clone the Hugging Face Space
if not os.path.exists("Ace-Step-v1.5"):
    print("Cloning ACE-Step v1.5 repository...")
    !git clone https://huggingface.co/spaces/ACE-Step/Ace-Step-v1.5
else:
    print("Repository already exists.")

# 2. Force Change Directory
# This ensures subsequent commands run inside the folder
os.chdir("/content/Ace-Step-v1.5")
print(f"Working directory set to: {os.getcwd()}")

# 3. Patch requirements.txt
# The original file asks for torch>=2.9.1 (future version) for Linux, which fails install.
# We relax this to allow the current stable torch version.
print("Patching requirements.txt...")
!sed -i 's/torch>=2.9.1/torch/g' requirements.txt

# 4. Patch app.py for Public Access
# By default, share=False. We change it to share=True to get a public Gradio link.
print("Patching app.py to enable public sharing...")
!sed -i 's/share=False/share=True/g' app.py

print("✅ Setup phase 1 complete.")

In [ ]:
# @title 2. Install Dependencies
import os

# Ensure we are in the correct folder
if os.getcwd() != "/content/Ace-Step-v1.5":
    os.chdir("/content/Ace-Step-v1.5")

print("Installing Python dependencies... (This may take 3-5 minutes)")

# Install dependencies with CUDA support
# We skip strict version checks for some libs to avoid Colab conflicts
!pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

# Install system dependency for audio processing
!apt-get install -y ffmpeg

print("✅ Dependencies installed.")

In [ ]:
# @title 3. Download Models & Launch Interface
# This cell first downloads the heavy model files (safetensors) and then starts the app.
import os

if os.getcwd() != "/content/Ace-Step-v1.5":
    os.chdir("/content/Ace-Step-v1.5")

# --- LITE MODE CONFIG ---
os.environ["SERVICE_MODE_DIT_MODEL_2"] = "" # Disable 2nd model download/load
os.environ["SERVICE_MODE_BACKEND"] = "vllm"

# 1. CREATE DOWNLOAD SCRIPT
# Note: Double backslashes (\\n) are used to prevent Python from breaking strings across lines in the file
download_code = """
import os
import sys
import torch

print("Initializing download sequence...")
current_dir = os.getcwd()
sys.path.insert(0, os.path.join(current_dir, "acestep", "third_parts", "nano-vllm"))

from acestep.handler import AceStepHandler

# Initialize handler to trigger download logic
# We force CPU offload to avoid filling GPU RAM just for downloading
handler = AceStepHandler(persistent_storage_path=os.path.join(current_dir, "data"))

print("Downloading ACE-Step v1.5 Unified Repo (This includes DiT and LLM)...")
print("This may take a while (approx 10GB)...")

config_path = "acestep-v15-turbo"

# Use internal method if available to force download, otherwise init
try:
    if hasattr(handler, '_ensure_model_downloaded'):
        handler._ensure_model_downloaded(current_dir, config_path)
        print("\\n✅ Download complete.")
    else:
        print("Standard initialization...")
        handler.initialize_service(current_dir, config_path, device='cpu', offload_to_cpu=True)
except Exception as e:
    print(f"\\nDownload process finished (check for errors above if any): {e}")
"""

with open("download_models.py", "w") as f:
    f.write(download_code)

# 2. RUN DOWNLOAD
print("⏳ Starting Model Download... (This prevents timeouts)")
!python download_models.py

# 3. LAUNCH APP
print("="*60)
print("🚀 Models ready. Launching ACE-Step Interface...")
print("🔗 Click the public link ending in 'gradio.live' below once it appears.")
print("="*60)

!python app.py